In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import seaborn as sns
import matplotlib.pyplot as plt

Load cleaned dataset

In [2]:
df_cleaned = pd.read_csv("/content/drive/MyDrive/account_transactions.csv")

Convert bookingDateTime to datetime format

In [5]:
df_cleaned["bookingDateTime"] = pd.to_datetime(df_cleaned["bookingDateTime"], errors="coerce")

Drop columns with excessive missing values

In [7]:
df_cleaned = df_cleaned.drop(columns=["merchant_logo", "merchant_merchantCategoryCode"], errors='ignore')

Fill missing values in 'merchant_name' with 'Unknown'

In [8]:
df_cleaned["merchant_name"].fillna("Unknown", inplace=True)

<ipython-input-8-5045e9aaa93c>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned["merchant_name"].fillna("Unknown", inplace=True)


Encode categorical variables

In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
categorical_cols = ["accountProductType", "providerId", "transactionDescription",
                    "category_group", "category_name", "creditDebitIndicator", "merchant_name"]

for col in categorical_cols:
    le = LabelEncoder()
    df_cleaned[col] = le.fit_transform(df_cleaned[col])
    label_encoders[col] = le

Extract useful time-based features

In [11]:
df_cleaned["booking_year"] = df_cleaned["bookingDateTime"].dt.year
df_cleaned["booking_month"] = df_cleaned["bookingDateTime"].dt.month
df_cleaned["booking_day"] = df_cleaned["bookingDateTime"].dt.day
df_cleaned["booking_weekday"] = df_cleaned["bookingDateTime"].dt.weekday

Log transform 'amount_value' to reduce impact of extreme values

In [12]:
df_cleaned["amount_value"] = df_cleaned["amount_value"].apply(lambda x: np.log1p(x))

Drop original bookingDateTime column

In [13]:
df_cleaned.drop(columns=["bookingDateTime"], inplace=True)

Initialize anomaly flag column

In [14]:
df_cleaned["anomaly_flag"] = 0

Train an Isolation Forest for each account individually

In [15]:
account_anomalies = {}

for account_id, group in df_cleaned.groupby("accountId"):
    if len(group) < 10:
        continue

    features = ["amount_value", "booking_year", "booking_month", "booking_day",
                "booking_weekday", "category_name", "creditDebitIndicator", "merchant_name"]

    X = group[features]

    iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
    group["anomaly_flag"] = iso_forest.fit_predict(X)

    # Convert -1 to 1 (anomaly) and 1 to 0 (normal)
    group["anomaly_flag"] = group["anomaly_flag"].apply(lambda x: 1 if x == -1 else 0)

    account_anomalies[account_id] = group[group["anomaly_flag"] == 1]

    df_cleaned.loc[group.index, "anomaly_flag"] = group["anomaly_flag"]

Save flagged transactions

In [16]:
df_cleaned.to_csv("account_specific_flagged_transactions.csv", index=False)

In [17]:
df_cleaned.head()

,transactionId,accountId,accountProductType,providerId,transactionDescription,category_group,category_name,category_icon,merchant_name,creditDebitIndicator,amount_value,amount_currency,booking_year,booking_month,booking_day,booking_weekday,anomaly_flag
0,8934c440-fd1b-4f4a-9b53-7a00d7f0a30d,5053cbd4-a32b-301d-b17f-aeb7ebc637aa,0,0,10,2,17,https://tg-merchants-prod.tarabut.com/category...,56,1,5.017280,SAR,2024,5,21,1,1
1,31f71281-b59c-4a14-92d1-4d96ba212ac7,5053cbd4-a32b-301d-b17f-aeb7ebc637aa,0,0,10,2,17,https://tg-merchants-prod.tarabut.com/category...,56,1,5.017280,SAR,2024,5,21,1,1
2,dd330de8-45e7-40ca-a012-d8078e7a87b6,5053cbd4-a32b-301d-b17f-aeb7ebc637aa,0,0,122,1,12,https://tg-merchants-prod.tarabut.com/category...,56,0,2.917771,SAR,2024,5,5,6,0
3,f135bf3c-04c2-45c6-b4fb-185cee532cd7,5053cbd4-a32b-301d-b17f-aeb7ebc637aa,0,0,53,1,12,https://tg-merchants-prod.tarabut.com/category...,56,0,0.009950,SAR,2024,8,4,6,1
4,f1265286-37b1-417d-9757-deead4d2e934,5053cbd4-a32b-301d-b17f-aeb7ebc637aa,0,0,53,1,12,https://tg-merchants-prod.tarabut.com/category...,56,0,0.009950,SAR,2024,8,4,6,1


In [21]:
df_cleaned["accountId"].unique()

array(['5053cbd4-a32b-301d-b17f-aeb7ebc637aa',
       '6549b0dd-27a1-3f5b-ab2a-9a83d8245d16',
       '9f556dd9-43ff-3569-b701-357236b21ed9',
       'c0df3c2f-b304-31b1-ad67-f5ec726ebf37',
       'c5950a78-122e-3fba-b8c7-8d43914bfe92',
       'ee229694-9a70-35da-ab0c-198fce6fe639'], dtype=object)

Count normal (0) and anomaly (1) transactions per account

In [23]:
account_summary = df_cleaned.groupby("accountId")["anomaly_flag"].value_counts().unstack(fill_value=0)
account_summary.columns = ["Normal Transactions", "Anomalous Transactions"]

print("\n🔹 Total Normal & Anomaly Transactions per Account:\n")
print(account_summary)


🔹 Total Normal & Anomaly Transactions per Account:

                                      Normal Transactions  \
accountId                                                   
5053cbd4-a32b-301d-b17f-aeb7ebc637aa                  193   
6549b0dd-27a1-3f5b-ab2a-9a83d8245d16                   88   
9f556dd9-43ff-3569-b701-357236b21ed9                   34   
c0df3c2f-b304-31b1-ad67-f5ec726ebf37                   14   
c5950a78-122e-3fba-b8c7-8d43914bfe92                   14   
ee229694-9a70-35da-ab0c-198fce6fe639                  109   

                                      Anomalous Transactions  
accountId                                                     
5053cbd4-a32b-301d-b17f-aeb7ebc637aa                      10  
6549b0dd-27a1-3f5b-ab2a-9a83d8245d16                       5  
9f556dd9-43ff-3569-b701-357236b21ed9                       2  
c0df3c2f-b304-31b1-ad67-f5ec726ebf37                       1  
c5950a78-122e-3fba-b8c7-8d43914bfe92                       1  
ee229694-9a70-35d